In [2]:
# imports

# own packages
from models import Base, Product, Store, Type_product, Contry, Currency, Rol, User, Base
from classDB import Connection, create_tables, check_keys

# external packages
from sqlalchemy import insert, update as Update, create_engine, inspect
from sqlalchemy.orm import sessionmaker
from sqlalchemy.sql import text
from dotenv import load_dotenv
import os
import pandas as pd
import ast

In [3]:
def create(model: str, data: list | dict = None, debug: bool = True) -> dict| bool | None:
    
    if debug:
        
        len_data = len(data)
        dict_error = []
        
        if type(data) is list:
            
            index_error = []
            
            for index, dt in enumerate(data):
                is_correct = check_keys(model = model, keys = dt.keys())
                if not(is_correct): 
                    index_error.append(index)
            
            if len(index_error) == len(data):
                print("El formato ingresado esta completamente malo verifica bien que este bien escrito el nombre de los campos")
                return False
            
            if len(index_error) >= 1:
                index_error.reverse()
                for index in index_error:
                    dict_error.append(data.pop(index))

                print(f"de {len_data}, {len(index_error)} estan malos, los cuales sus index son: {index_error}")
                print(f"del resto {len(data)} su estructura esta bien asi que se procedera a crear en la base de datos.")

                
        else: 
            is_correct = check_keys(model = model, keys = data.keys())
            if not(is_correct):
                return is_correct


    connection = Connection()
    with Connection.engine.connect() as con:
        
        try:
            connection.session.execute(insert(model), data)
            model.__str__(data = data)
        except TypeError as e:
            model.__str__(data = data, error = False)
            print(e)
                
        connection.session.commit()
    
    if debug:
    
        if len(dict_error) >= 1:
            print("Se te va a retornar los diccionarios que estan mal estructurados para que los puedas arreglas e ingresar de nuevo")
            return dict_error

In [7]:
def read(model: str, search : dict = None, debug = True, and_or: str = "and") -> list | dict:
    
    
    if debug:
        params_search = list(search.keys())

        if search != None:
            if not(check_keys(model=model, keys = params_search)):
                print("La busqueda que esta realizando esta mal escrita o mal estructurada, intente de nuevo.")
                return
    
    connection = Connection()
    with connection.engine.connect() as con:
        
        if search == None: # si no se le astablece un paramtro de busqueda listara todos los elementos de la tabla y retornara un dataFrame
            data = connection.session.execute(text(f"SELECT * FROM {model.__tablename__}")).fetchall()
            data_list = []
            for index in range(0, len(data)):
                data_dict = {key:value for key,value in zip(model.__column__, data[index])}
                data_list.append(data_dict)
            return pd.DataFrame(data_list)
        
        else:
            
            querys = []
            
            if type(search) is dict:
                params_search = list(search.keys())
                params_value = list(search.values())
                query = f"SELECT * FROM {model.__tablename__} WHERE "
                
                if len(params_search) >= 2:
                    for search, value in zip(params_search, params_value):
                        if type(value) is str: query+= f"{and_or} {search} = '{value}'"
                        elif type(value) is int: query+= f"{and_or} {search} = {value}"
                        elif type(value) is bool: query+= f"{and_or} {search} = {value}"
                        querys.append(query)
                else:
                    if type(params_value[0]) is str: query+= f" {params_search[0]} = '{params_value[0]}'"
                    elif type(params_value[0]) is int: query+= f" {params_search[0]} = {params_value[0]}"
                    elif type(params_value[0]) is bool: query+= f" {params_search[0]} = {params_value[0]}"
                    querys.append(query)
                    
            elif type(search) is list:
                searchs = search
                for search in searchs:
                    if type(search) is dict:
                        params_search = list(search.keys())
                        params_value = list(search.values())
                        query = f"SELECT * FROM {model.__tablename__} WHERE "
                        
                        if len(params_search) >= 2:
                            for search, value in zip(params_search, params_value):
                                if type(value) is str: query+= f"{and_or} {search} = '{value}'"
                                elif type(value) is int: query+= f"{and_or} {search} = {value}"
                                elif type(value) is bool: query+= f"{and_or} {search} = {value}"
                                querys.append(query)
                        else:
                            if type(params_value[0]) is str: query+= f" {params_search[0]} = '{params_value[0]}'"
                            elif type(params_value[0]) is int: query+= f" {params_search[0]} = {params_value[0]}"
                            elif type(params_value[0]) is bool: query+= f" {params_search[0]} = {params_value[0]}"
                            querys.append(query)
            
            list_search = []
            for query in querys:
                datos = connection.session.execute(text(query)).fetchall()
                
                if len(datos) == 0:
                    print("Lo que esta buscando no existe o esta mal escrito, intente de nuevo")
                    list_search.append(False)
                else:
                    for data in datos:
                        data_dict = {key:value for key,value in zip(model.__column__, data)}
                        list_search.append(data_dict)
                        
                        
            return list_search

In [14]:
def update(model: str, param: dict | list):
    
    if not(check_keys(model=model, keys=list(param.keys()))):
        print("La peticion ingresada, esta mal escrita, o no existe el paramtro al que se esta consultando.")
        return
    
    model_id, model_unique = model.__column__[0], model.__unique__
    
    conection = Connection()
    with conection.engine.connect() as con:
        
        if type(param) is dict:
            keys_param = list(param.keys())
            
            if model_id in keys_param:
                conection.session.execute(Update(model), param)
                conection.session.commit()
                print("Actualizado")
                
            elif model_unique in keys_param:
                data = read(model=model, search={model_unique:param[model_unique]})
                param[model_id] = data[0][model_id]
                conection.session.execute(Update(model), param)
                conection.session.commit()
                print("Actualizado")
                
        elif type(param) is list:
            
            params = param
            for param in params:
                if model_id in keys_param:
                    conection.session.execute(Update(model), param)
                    conection.session.commit()
                    print("Actualizado")
                    
                elif model_unique in keys_param:
                    data = read(model=model, search={model_unique:param[model_unique]})
                    param[model_id] = data[0][model_id]
                    conection.session.execute(Update(model), param)
                    conection.session.commit()
                    print("Actualizado")